<a href="https://colab.research.google.com/github/goergevo/FreeRTOSxMegaPortAndDemo/blob/master/TellTales_MobilenetV2_multilabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MobileNet_V2 **multilabel** transfer learning.
Test with CPU or GPU.

To keep the backend running:
in Chrome: alt-cmd I
and insert this code:
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)

In [0]:
!pip install -q livelossplot

How much memory does my GPU have?

In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
try:
  gpu = GPUs[0]
  def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  printm()
except:
  print("no GPU!")
  pass

no GPU!


In [3]:
# Download and install the TensorFlow 2.0
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  print("no Tensorflow 2.0 available?")
  pass

TensorFlow 2.x selected.


In [4]:
# mount my personal google drive
from google.colab import drive
# mount under this path in the local runtime:
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!jupyter nbconvert --to script "drive/My Drive/Colab Notebooks/TellTales_MobilenetV2_multilabel.ipynb"

/tensorflow-2.1.0/python3.6/pkg_resources/py2_warn.py:19: UserWarning: ************************************************************
You are running Setuptools on Python 2, which is no longer
supported and
>>> SETUPTOOLS WILL STOP WORKING <<<
in a subsequent release. Please ensure you are installing
Setuptools using pip 9.x or later or pin to `setuptools<45`
in your environment.
If you have done those things and are still encountering
this message, please comment in
https://github.com/pypa/setuptools/issues/1458
about the steps that led to this unsupported combination.
************************************************************
  sys.version_info < (3,) and warnings.warn("*" * 60 + msg + "*" * 60)
[NbConvertApp] Converting notebook drive/My Drive/Colab Notebooks/TellTales_MobilenetV2_multilabel.ipynb to script
[NbConvertApp] Writing 31790 bytes to drive/My Drive/Colab Notebooks/TellTales_MobilenetV2_multilabel.txt


In [6]:
!ls "/content/drive/My Drive"
!cp "/content/drive/My Drive/imagesRedGreen2_1000_128x128.zip" .
#!rm -r imagesRedGreen200
#!rm -r imagesRedGreen1000
!unzip -q -u imagesRedGreen2_1000_128x128.zip
!ls -l

'Colab Notebooks'
'How to get started with Drive.pdf'
 imagesRedGreen1000_112x112.zip
 imagesRedGreen200_112x112.zip
 imagesRedGreen20_112x112.zip
 imagesRedGreen2_1000_128x128.zip
 traindir_20191220_170054_gut_97prozent
 traindir_20191220_232930_gut_98prozent
 traindir_20191222_173741_gut_99prozent
 traindir_20191223_172537_alpha1.0_gut_99.9prozent
 traindir_20191224_223926_alpha0.5_gut_99.5prozent
 traindir_20191225_074057_alpha0.35_gut_98.9prozent
 traindir_20191226_144336_alpha0.35_99prozent
 traindir_20200129_234140_alpha0.35_97.55prozent
 traindir_20200129_235150_alpha1.0_98.42prozent
 traindir_20200130_123444_aplha1.0_99.35prozent
 traindir_20200131_121503
total 146960
-rw-r--r-- 1 root root   5102824 Jan 31 21:41 df_train.csv
-rw-r--r-- 1 root root   1007665 Jan 31 21:41 df_valid.csv
drwx------ 4 root root      4096 Jan 31 21:37 drive
drwxr-xr-x 4 root root      4096 Jan 31 17:09 imagesRedGreen2_1000_128x128
-rw------- 1 root root 144342334 Jan 31 22:00 imagesRedGreen2_1000_128

In [7]:
#  ***Multilabel***
#  Transfer Learning with MobileNet_V2
# tutorial: https://www.tensorflow.org/tutorials/images/transfer_learning
#
# https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e
# https://medium.com/@ckyeungac/a-simple-trick-about-multi-label-image-classification-with-imagedatagenerator-in-keras-4ab95364a481
# https://github.com/aditya9898/transfer-learning/blob/master/transfer-learning.py
#
# The building of a model is a 3 step process:
#
#    Importing the pre-trained model and adding the dense layers.
#    Loading train data into ImageDataGenerators.
#    Training and Evaluating model.
#
import pandas as pd
import numpy as np
#import glob
import pickle
import datetime
import os
import pathlib
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras  # we are using keras which is part of tensorflow 2.0
from sklearn.preprocessing import MultiLabelBinarizer
from livelossplot.keras import PlotLossesCallback

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# more info on callbacks: https://keras.io/callbacks/ model saver is cool too.

print("Keras Version=" + keras.__version__)  # expecting 2.2.4-tf
print("Tensorflow Version=" + tf.__version__)  # expecting Tensorflow version=2.0.0

NAME = "TellTales-CNN"

# The helper function
def multilabel_flow_from_dataframe(data_generator, mlb, df):
    for x, y in data_generator:
        # num_samples = len(y)
        indices = y.astype(np.int).tolist()
        y_multi = mlb.transform(df.iloc[indices]['tags'].values.tolist())
        yield x, y_multi


def createDataFrame_fromDirectory(images_root_dir):
    df = pd.DataFrame({'imagepath': [], 'tags': [], 'index': []})
    i = int(0)
    for filename in images_root_dir.rglob('*/*.jpg'):
        # print(filename)
        # the directory name contains the angle combinations: .../degree_45_15/xy.png
        labels = filename.parts[-2].split("_")
        df = df.append({'imagepath': str(filename), 'tags': labels, 'index': i}, ignore_index=True)
        i += 1
    #print(df)
    return df


Keras Version=2.2.4-tf
Tensorflow Version=2.1.0


Using TensorFlow backend.


In [0]:
# ============================================================================
#
class trainMobileNetV2_MultiLabel:
    def __init__(self):
        self.width = 0
        self.height = 0
        self.model_data_name = ""
        self.mlb_data_name = ""
        self.numclasses = 0
        self.model_plot_name = 'model.png'
        self.number_of_epochs = 0
        self.model = 0
        self.history = 0
        self.root_dir_train = 0
        self.root_dir_valid = 0
        self.df_train = 0
        self.df_valid = 0
        self.mlb_train = 0
        self.mlb_valid = 0
        self.classes_list = 0

    # ============================================================
    #
    def enumerate_training_images(self):
        # parse the images directories and create the dataframe with images pathes and label lists
        self.df_train = createDataFrame_fromDirectory(self.root_dir_train)
        self.df_valid = createDataFrame_fromDirectory(self.root_dir_valid)

        self.df_train.to_csv("df_train.csv", index=False)
        self.df_valid.to_csv("df_valid.csv", index=False)

        # Fit a MultiLabelBinarizer
        self.mlb_train = MultiLabelBinarizer()
        self.mlb_train.fit(self.df_train['tags'].values.tolist())
        self.mlb_valid = MultiLabelBinarizer()
        self.mlb_valid.fit(self.df_valid['tags'].values.tolist())

        # use this later for the predictions
        self.classes_list = self.mlb_train.classes_
        self.numclasses = len(self.classes_list)
        print("found this image classes in training data:")
        print(self.classes_list)

    # ============================================================
    #
    def build_model(self, num_fix_layers, learning_rate, alpha_param):
        # We import the MobileNet model without its last layer ...
        # weights are downloaded and cached here: ~/.keras/models/mobilenet_1_0_224_tf_no_top.h5
        #
        # https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
        # The paper demonstrates the performance of MobileNets using `alpha` values of
        # 1.0 (also called 100 % MobileNet), 0.35, 0.5, 0.75, 1.0, 1.3, and 1.4
        #
        # For each of these `alpha` values, weights for 5 different input image sizes
        # are provided (224, 192, 160, 128, and 96).
        #
        base_model = keras.applications.MobileNetV2(input_shape=(self.width, self.height, 3),
                                                    alpha= alpha_param,
                                                    #weights='imagenet',# imports the trained mobilenet weights
                                                    include_top=False)  # and discards the last 1000 neuron layer.

        #base_model.trainable = False

        # ... and add a few dense layers so that our model can learn more complex functions. The dense layers must
        # have the relu activation function and the last layer,which contains as many neurons as the number of
        # classes must have the softmax activation.
        x = base_model.output
        x = keras.layers.GlobalAveragePooling2D()(x)
        # we add dense layers so that the model can learn more complex functions and
        # classify for better results.
        #todo: make it a parameter
        #x = keras.layers.Dense(1280, activation='relu')(x)
        #x = keras.layers.Dense(1280, activation='relu')(x)
        x = keras.layers.Dense(512, activation='relu')(x)
        # final layer with sigmoid activation for multilabel classification
        predictions = keras.layers.Dense(self.numclasses, activation='sigmoid')(x)

        # Next we make a model based on the architecture we have provided.
        # specify the inputs
        # specify the outputs
        self.model = keras.models.Model(inputs=base_model.input, outputs=predictions)
        # now a model has been created based on our architecture

        # Now that we have our model, as we will be using the pre-trained weights, that our model has been trained on (
        # imagenet dataset), we have to set all the weights to be non-trainable. We will only be training the last Dense
        # layers that we have made previously.

        for layer in self.model.layers[:num_fix_layers]:
            layer.trainable = False
        for layer in self.model.layers[num_fix_layers:]:
            layer.trainable = True

        # or: all layers non-trainable:
        # for layer in model.layers:
        #    layer.trainable=False

        # To check the architecture of our model, we simply need to use this line of code given below.
        #for i, layer in enumerate(self.model.layers):
        #    print(i, layer.name, layer.trainable)
        
        # another method to check the model structure:
        #self.model.summary()

        # For this we first compile the model that we made, and then train our model with our generator:
        #   Adam optimizer
        #   loss function will be binary crossentropy for multi-label classification
        #   evaluation metric will be accuracy
        opt = keras.optimizers.Adam(lr=learning_rate)
        # todo: learning rate decay
        # opt = keras.optimizers.Adam(lr=learning_rate, decay=learning_rate / number_of_epochs)
        self.model.compile(optimizer=opt,
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        # create a graphical plot of the model
        #  keras.utils.plot_model(self.model, to_file=self.model_plot_name)

    # ============================================================
    #
    def train(self, number_of_epochs, batch_size):

        train_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)
        train_generator = train_datagen.flow_from_dataframe(
            dataframe=self.df_train,
            directory='./',
            x_col='imagepath',
            y_col='index',
            class_mode='other',
            target_size=(self.width, self.height),
            color_mode="rgb",
            batch_size=batch_size
        )
        multilabel_train_generator = multilabel_flow_from_dataframe(train_generator, self.mlb_train, self.df_train)

        # validate the model with unseen images
        valid_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)
        valid_generator = valid_datagen.flow_from_dataframe(
            dataframe=self.df_valid,
            directory='./',
            x_col='imagepath',  # column names in the df
            y_col='index',
            class_mode='other',
            target_size=(self.width, self.height),
            color_mode="rgb",
            batch_size=batch_size
        )
        multilabel_valid_generator = multilabel_flow_from_dataframe(valid_generator, self.mlb_valid, self.df_valid)

        # log decreasing learning rate for Tensorboard
        #
        logdir = "logs/scalars/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        file_writer = tf.summary.create_file_writer(logdir + "/metrics")
        file_writer.set_as_default()
        def lr_schedule(epoch, lr):
            """
            Returns a custom learning rate that decreases as epochs progress.
            """
            learning_rate = lr
            #learning_rate = 0.01
            if epoch > 5:
                learning_rate /= 2
            if epoch > 8:
                learning_rate /= 4
            if epoch > 11:
                learning_rate /= 8

            tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
            return learning_rate

        plot_losses = PlotLossesCallback()
        #logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        #tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
        lr_callback        = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

        step_size_train = train_generator.n // train_generator.batch_size
        step_size_valid = valid_generator.n // valid_generator.batch_size
        self.number_of_epochs = number_of_epochs
        history = self.model.fit_generator( generator=multilabel_train_generator,
                                  steps_per_epoch=step_size_train,
                                  epochs=number_of_epochs,
                                  validation_data=multilabel_valid_generator,
                                  validation_steps=step_size_valid,
                                  # total number of validation batches to perform on an epoch
                                  validation_freq=1,
                                  # how many training epochs to run before a new validation run is performed
                                  callbacks=[plot_losses, lr_callback],
                                  #callbacks=[tensorboard_callback, lr_callback],
                                  verbose=1)  # 0=silent, 1 = progress bar, 2 = one line per epoch
        # Now we have a trained model.
        self.history = history

        print('===== history data ======')
        print(history.history.keys())
        print('=========================')

        # This saves the model exactly in the state where it was stopped.
        # This model can be used to continue training.
        self.model.save(str(self.model_data_name))

        # save the multi-label binarizer to disk
        # todo: save mlb_train as well?
        print("[INFO] serializing label binarizer...")
        f = open(self.mlb_data_name, "wb")
        f.write(pickle.dumps(self.mlb_valid))
        f.close()

        print("Learning done.")

    # =================================
    # plot accuracy and loss
    #
    def plotAccuracy(self):
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        t = f.suptitle('Basic CNN Performance', fontsize=12)
        f.subplots_adjust(top=0.85, wspace=0.3)

        epoch_list = list(range(1, self.number_of_epochs + 1))
        ax1.plot(epoch_list, self.history.history['accuracy'], label='Train Accuracy')
        ax1.plot(epoch_list, self.history.history['val_accuracy'], label='Validation Accuracy')
        ax1.set_xticks(np.arange(0, self.number_of_epochs + 1, 5))
        ax1.set_ylabel('Accuracy Value')
        ax1.set_xlabel('Epoch')
        ax1.set_title('Accuracy')
        l1 = ax1.legend(loc="best")

        ax2.plot(epoch_list, self.history.history['loss'], label='Train Loss')
        ax2.plot(epoch_list, self.history.history['val_loss'], label='Validation Loss')
        ax2.set_xticks(np.arange(0, self.number_of_epochs + 1, 5))
        ax2.set_ylabel('Loss Value')
        ax2.set_xlabel('Epoch')
        ax2.set_title('Loss')
        l2 = ax2.legend(loc="best")
        plt.show()

        k = cv2.waitKey(0) & 0xFF
        if k == 27:  # wait for ESC key to exit
            cv2.destroyAllWindows()
        elif k == ord('s'):  # wait for 's' key to save and exit
            cv2.destroyAllWindows()

    # =================================
    # plot accuracy and loss
    #
    def plot2(self, traindatadir):
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        t = f.suptitle('Basic CNN Performance', fontsize=12)
        f.subplots_adjust(top=0.85, wspace=0.3)

        epoch_list = list(range(1, self.number_of_epochs + 1))
        # ax1.plot(epoch_list, self.history.history['acc'], label='Train Accuracy')
        # ax1.plot(epoch_list, self.history.history['val_acc'], label='Validation Accuracy')
        ax1.plot(epoch_list, self.history.history['accuracy'], label='Train Accuracy')
        ax1.plot(epoch_list, self.history.history['val_accuracy'], label='Validation Accuracy')
        ax1.set_xticks(np.arange(0, self.number_of_epochs + 1, 5))
        ax1.set_ylabel('Accuracy Value')
        ax1.set_xlabel('Epoch')
        ax1.set_title('Accuracy')
        l1 = ax1.legend(loc="best")

        ax2.plot(epoch_list, self.history.history['loss'], label='Train Loss')
        ax2.plot(epoch_list, self.history.history['val_loss'], label='Validation Loss')
        ax2.set_xticks(np.arange(0, self.number_of_epochs + 1, 5))
        ax2.set_ylabel('Loss Value')
        ax2.set_xlabel('Epoch')
        ax2.set_title('Loss')
        l2 = ax2.legend(loc="best")
        plt.savefig(str(traindatadir / "lossplot"))
        # plt.show()

    # =================================
    # create and store the loss plot diagram
    #
    def plot(self, traindatadir, numepochs):
        # plot the training loss and accuracy
        N = numepochs
        H = self.history
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
        plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
        plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
        plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
        plt.title("Training Loss and Accuracy on Dataset")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend(loc="lower left")
        plt.savefig(traindatadir + "lossplot")



In [0]:
# ============================================================================
#
class testMobileNetV2_MultiLabel:

    def __init__(self):
        self.model = 0
        self.df_test = 0  # dataframe
        self.model_data_name = ""
        self.mlb_test = 0
        self.width = 0
        self.height = 0
        self.root_dir_test = ""
        self.resultstring = ""

    def enumerate_test_images(self):
        # parse the images directories and create the dataframe with images pathes and label lists
        print(self.root_dir_test)
        self.df_test = createDataFrame_fromDirectory(self.root_dir_test)
        # Fit a MultiLabelBinarizer
        self.mlb_test = MultiLabelBinarizer()
        self.mlb_test.fit(self.df_test['tags'].values.tolist())
        print("found this image classes in test data:")
        print(self.mlb_test.classes_)

    def buildModel(self):
        # load to continue training?
        #
        # even a compile is unnecessary
        # https://www.tensorflow.org/tutorials/distribute/save_and_load
        self.model = tf.keras.models.load_model(str(self.model_data_name))

        # for inference:
        #DEFAULT_FUNCTION_KEY = "serving_default"
        #loaded = tf.saved_model.load(str(self.model_data_name))
        #inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]


    # =================================
    # test with test data
    #
    def testing(self):

        # the model with unseen images
        print("Testing with test_generator ...")

        test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)
        test_generator = test_datagen.flow_from_dataframe(
            dataframe=self.df_test,
            directory='./',
            x_col='imagepath',  # column names in the df
            y_col='index',
            class_mode='other',
            target_size=(self.width, self.height),
            color_mode="rgb",
            batch_size=1,   # each image exactly once
            shuffle=False)  # test sequence in order

        multilabel_test_generator = multilabel_flow_from_dataframe(test_generator, self.mlb_test, self.df_test)
        STEP_SIZE_TEST = test_generator.n // test_generator.batch_size
        test_generator.reset()
        predictions = self.model.predict_generator(multilabel_test_generator,
                                                   steps=STEP_SIZE_TEST,
                                                   verbose=0)  # 0=silent, 1 = progress bar, 2 = one line per epoch

        # predicted_class_indices = np.argmax(pred, axis=1)
        # labels = test_generator.class_indices
        # labels = dict((v, k) for k, v in labels.items())
        # predictions = [labels[k] for k in predicted_class_indices]

        filenames = test_generator.filenames  # contains each test files only once

        top2_probabilities_list = []
        number_of_correct_predictions = 0
        for (filename, probabilities_list) in zip(filenames, predictions):
            predictionstring = filename
            sortet_propability_indexes = np.argsort(probabilities_list)[::-1]
            propability_index_0 = sortet_propability_indexes[0]
            propability_index_1 = sortet_propability_indexes[1]
            label0 = self.mlb_test.classes_[propability_index_0]
            label1 = self.mlb_test.classes_[propability_index_1]
            if label0[0] == 'G':  # we want Red left and Green on right
                x = label0
                label0 = label1
                label1 = x
            # propability_percent_val_0 = probabilities_list[propability_index_0] * 100
            # propability_percent_val_1 = probabilities_list[propability_index_1] * 100
            top2_probabilities_list.append([label0, label1])

            expected_label0 = os.path.split(os.path.dirname(filename))[-1].split('_')[0]
            expected_label1 = os.path.split(os.path.dirname(filename))[-1].split('_')[1]
            expected_angle0 = ''.join(i for i in expected_label0 if i.isdigit())
            expected_angle1 = ''.join(i for i in expected_label1 if i.isdigit())
            pred_angle0 = ''.join(i for i in label0 if i.isdigit())  # extract number
            pred_angle1 = ''.join(i for i in label1 if i.isdigit())  # extract number

            if (expected_angle0 == pred_angle0):
                number_of_correct_predictions += 1
            if (expected_angle1 == pred_angle1):
                number_of_correct_predictions += 1

            # predictionstring += " {}: {:.2f}% , {}: {:.2f}%".format(label0, propability_percent_val_0, label1, propability_percent_val_1)
            # print(predictionstring)

        all_predictions = 2 * len(filenames)
        number_of_correct_predictions_percent = number_of_correct_predictions / (all_predictions) * 100
        self.resultstring = "correct predictions: {} of {} = {:.2f}%".format(number_of_correct_predictions,
                                                                             all_predictions,
                                                                             number_of_correct_predictions_percent)
        print(self.resultstring)

        results = pd.DataFrame({"Filename": filenames, "Predictions": top2_probabilities_list})
        results.to_csv(self.resultscsv_name, index=False)

        print("Done testing. Results written to results.csv")

    # =================================
    # The trained model can now be used to predict which class a new unseen
    # image belongs to, by using model.predict(new_image).
    #
    def predict(self, img_path):
        print("predicting: " + img_path)
        img = keras.preprocessing.image.load_img(img_path, target_size=(self.width, self.height))
        image = keras.preprocessing.image.img_to_array(img)
        image = np.expand_dims(image, axis=0)
        image = keras.applications.mobilenet.preprocess_input(image)
        probabilities = self.model.predict(image)[0]
        idxs = np.argsort(probabilities)[::-1][:2]
        # print("len: " + str(len(idxs)))

        # loop over the indexes of the high confidence class labels
        print("Prediction:")
        for (i, j) in enumerate(idxs):
            # print the two labels for Red and Green thread:
            label = "{}: {:.2f}%".format(self.mlb_test.classes_[j], probabilities[j] * 100)
            print(label)

        # show the probabilities for each of the individual labels
        print("Details:")
        for (label, p) in zip(self.mlb_test.classes_, probabilities):
            print("{}: {:.2f}%".format(label, p * 100))

    # ============================================================================
    #
    def predictAll(self, root_dir_test):
        image_filename_list = self.df_test.iloc[:]['imagepath']
        number_of_correct_predictions = 0
        number_of_nearly_correct_predictions = 0
        for imagefile in image_filename_list:
            img = keras.preprocessing.image.load_img(imagefile, target_size=(self.width, self.height))
            image = keras.preprocessing.image.img_to_array(img)
            image = np.expand_dims(image, axis=0)
            image = keras.applications.mobilenet.preprocess_input(image)
            # ****
            probabilities_list = self.model.predict(image)[0]  # inference of single image
            # ****
            probability_index_green = np.argsort(probabilities_list[:7])[::-1][0]  # 0..6
            probability_index_red   = np.argsort(probabilities_list[7:])[::-1][0]  # 7..13
            label_green = self.mlb_test.classes_[probability_index_green]
            label_red   = self.mlb_test.classes_[probability_index_red + 7]
            if label_red[0] == 'G' or label_green[0] == 'R':  # we want Red in label_red and Green in label_green
                print("unexpected label type: should not happen!")

            expected_label_red = os.path.split(os.path.dirname(imagefile))[-1].split('_')[0]  # RED
            expected_label_green = os.path.split(os.path.dirname(imagefile))[-1].split('_')[1]  # GREEN

            # RAND=999
            label_red            = label_red.replace("RAND", "999")
            label_green          = label_green.replace("RAND", "999")
            expected_label_red   = expected_label_red.replace("RAND", "999")
            expected_label_green = expected_label_green.replace("RAND", "999")

            # extract number from labelname
            expected_angle_red = int(''.join(i for i in expected_label_red if i.isdigit()))
            expected_angle_green = int(''.join(i for i in expected_label_green if i.isdigit()))
            pred_angle_red = int(''.join(i for i in label_red if i.isdigit()))
            pred_angle_green = int(''.join(i for i in label_green if i.isdigit()))

            tolerance = 16
            if expected_angle_green == pred_angle_green:
                number_of_correct_predictions += 1
            elif (pred_angle_green >= (expected_angle_green - tolerance)) and (pred_angle_green <= (expected_angle_green + tolerance)):
                number_of_nearly_correct_predictions += 1
            else:
                print(imagefile)
                print("expected angle green:", expected_angle_green, "got:", pred_angle_green)

            if expected_angle_red == pred_angle_red:
                number_of_correct_predictions += 1
            elif (pred_angle_red >= (expected_angle_red - tolerance)) and (pred_angle_red <= (expected_angle_red + tolerance)):
                number_of_nearly_correct_predictions += 1
            else:
                print(imagefile)
                print("expected angle red:", expected_angle_red, "got:", pred_angle_red)

        all_predictions = 2 * len(image_filename_list)
        number_of_correct_predictions_percent = number_of_correct_predictions / all_predictions * 100
        self.resultstring = "correct predictions: {} of {} = {:.2f}%".format(number_of_correct_predictions,
                                                                             all_predictions,
                                                                             number_of_correct_predictions_percent)
        print("predictAll: " + self.resultstring)

        number_of_nearly_correct_predictions += number_of_correct_predictions
        number_of_nearly_correct_predictions_percent = number_of_nearly_correct_predictions / all_predictions * 100
        self.resultstringnear = "nearly correct: {} of {} = {:.2f}%".format(number_of_nearly_correct_predictions,
                                                                            all_predictions,
                                                                            number_of_nearly_correct_predictions_percent)
        print("predictAll: " + self.resultstringnear)


In [0]:
#%tensorboard --logdir logs

In [0]:
# ============================================================================
#
if __name__ == '__main__':
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    all_train_data_dir = pathlib.Path('traindir_' + current_time)

    images_root_dir = pathlib.Path('imagesRedGreen2_1000_128x128')

    doTrain = True
    doTest = True

    if doTrain:
        pathlib.Path(all_train_data_dir).mkdir(parents=True, exist_ok=True)

    # hyper parameter:
    width_height  = 128   # possible values: 96, 128, 160, 192, 224
    learning_rate = 0.001 # start with
    epochs        = 15
    fixed_layers  = 0     # non-trainable
    alpha         = 1.0   # possible values: 0.35, 0.5, 0.75, 1.0, 1.3, and 1.4

    if doTrain:
        print("start training ...")
        cnn = trainMobileNetV2_MultiLabel()
        cnn.root_dir_train = images_root_dir / 'train'
        cnn.root_dir_valid = images_root_dir / 'valid'
        cnn.root_dir_test = images_root_dir / 'valid'

        cnn.plot_name = all_train_data_dir / 'model_plot.png'
        cnn.model_data_name = all_train_data_dir / 'saved_model_state.model'
        cnn.mlb_data_name = all_train_data_dir / 'mlb.pickle'
        cnn.width = width_height  # possible values: 96, 128, 160, 192, 224
        cnn.height = width_height

        print("enumerating images ...")
        cnn.enumerate_training_images()

        cnn.build_model(fixed_layers, learning_rate, alpha)

        batch_size = 32
        cnn.train(epochs, batch_size)
        numclasses = len(cnn.classes_list)

        # plot accuracy and loss and save diagram
        cnn.plot2(all_train_data_dir)

        print("training Done.")

    # ===================== testing =================================================
    #
    if doTest:
        print("start testing ...")

        if not doTrain:
            # use pretrained model:
            all_train_data_dir = pathlib.Path('traindir_20191219_154722')

        cnntest = testMobileNetV2_MultiLabel()
        cnntest.width = width_height  # possible values: 96, 128, 160, 192, 224
        cnntest.height = width_height

        cnntest.model_data_name = all_train_data_dir / 'saved_model_state.model'
        cnntest.mlb_data_name = all_train_data_dir / 'mlb.pickle'
        cnntest.resultscsv_name = all_train_data_dir / 'results.csv'

        cnntest.root_dir_test = images_root_dir / 'valid'

        cnntest.enumerate_test_images()

        cnntest.buildModel()

        # img_path = './imagesRedGreen200/valid/RED15_GREEN345/telltale1_20_349.png'
        # cnntest.predict(img_path)

        cnntest.predictAll(all_train_data_dir)

        cnntest.testing()

        print("testing Done.")

    # document all training parameters:
    with open(all_train_data_dir / "trainingParameters.txt", "a+") as text_file:
        if doTrain:
            print("======== Training and Testing =======")
        else:
            print("======== Testing =======")
        print(f"Training Params of today {current_time}", file=text_file)
        print(f"Image witdth/height:     {width_height}", file=text_file)
        print(f"Training data:           {str(images_root_dir)}", file=text_file)
        if doTrain:
            print(f"Number of fix layers:    {fixed_layers}", file=text_file)
            print(f"Number of epochs:        {epochs}", file=text_file)
            print(f"Number of categories:    {numclasses}", file=text_file)
            print(f"learning_rate:           {learning_rate}", file=text_file)
            print(f"Batch  size:             {batch_size}", file=text_file)
            print(f"alpha:                   {alpha}", file=text_file)
            print(f"nur ein einziges dense512 Layer", file=text_file)
            print(f"keine vortrainierten Gewichte", file=text_file)
        print(f"test results: {cnntest.resultstring}", file=text_file)
        print(f"test results: {cnntest.resultstringnear}", file=text_file)
        text_file.close()

    if IN_COLAB:
        shutil.copytree(str(all_train_data_dir), '/content/drive/My Drive/' + str(all_train_data_dir))

    print("All Done.")


start training ...
enumerating images ...
